In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
from helper import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']

In [2]:
def generator(noise_dim=186, input_classes=3, out_dim=186):
    
    in_label = Input(shape=(1,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(noise_dim)(x)
    x = Reshape((noise_dim,1))(x)
    
    G_in = Input(shape=[noise_dim,])
    gen = Dense(noise_dim)(G_in)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((noise_dim,1))(gen)

    x = Concatenate()([gen, x])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=15, strides=1, padding='valid', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('tanh')(x)
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=100, input_classes=3, out_dim=186)
# plot_model(g_model, to_file='gen.png', show_shapes=True)

In [3]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [4]:
def load_real_samples():
#     X = np.load('Data/ForGAN/X.npy')
#     y = np.load('Data/ForGAN/y.npy')

    X = reshape(np.load('Data/TrainVal/X.npy'))
    y = np.load('Data/TrainVal/y.npy')

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [5]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    # np.random.shuffle(labels_fake)
    return [reshape(X_fake), labels_fake]

In [6]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [7]:
def show_plot(z_input, n_batch):
    X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    plt.show()
    # plt.savefig(name, dpi=75)
    plt.close()

In [8]:
def augment_using_GAN(g_model, Actual_Data, latent_dim, Samples, Batch_Size, label):
    X_Aug = []
    for i in range(0, Samples+Batch_Size, Batch_Size):
        
        print (i, end='\r')
        
        [z_input, _] = generate_latent_points(latent_dim, Batch_Size)
        
        if label==0:
            labels_input = np.zeros(Batch_Size)
        if label==1:
            labels_input = np.ones(Batch_Size)
        if label==2:
            labels_input = 2*np.ones(Batch_Size)
        
        z_input  = g_model.predict([z_input, labels_input], verbose=0)
        X_Aug.append(z_input)
    
    X_Aug = np.array(X_Aug)
    X_Aug = X_Aug.reshape(X_Aug.shape[0]*X_Aug.shape[1], X_Aug.shape[2], X_Aug.shape[3])
    X_Aug = np.vstack((Actual_Data, X_Aug))
    
    if label==0:
        y_Aug = np.zeros(X_Aug.shape[0])
    if label==1:
        y_Aug = np.ones(X_Aug.shape[0])
    if label==2:
        y_Aug = 2*np.ones(X_Aug.shape[0])
    
    return X_Aug, y_Aug

In [9]:
def combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen):
    g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
    g_model.load_weights(model_name + str(batch_no) + '_cgan_generator.h5')

    X_NN, y_NN = augment_using_GAN(g_model, X_N, latent_dim, N_Gen, n_batch, label=0)
    X_SS, y_SS = augment_using_GAN(g_model, X_S, latent_dim, S_Gen, n_batch, label=1)
    X_VV, y_VV = augment_using_GAN(g_model, X_V, latent_dim, V_Gen, n_batch, label=2)
    
    X = np.vstack((X_NN, X_SS, X_VV))
    y = np.hstack((y_NN, y_SS, y_VV))
    print (X.shape, y.shape)
    
    X_name = 'Data/GAN_GEN/' + batch_no + '_X'   
    y_name = 'Data/GAN_GEN/' + batch_no + '_y'   
    # print (X_name, y_name)
    
    np.save(X_name, X)
    np.save(y_name, y)

In [10]:
# X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# print (X_N.shape, y_N.shape, X_S.shape, y_S.shape, X_V.shape, y_V.shape)

X_N, _, X_S, _, X_V, _ = load_real_samples()
print (X_N.shape, X_S.shape, X_V.shape)

Total = 130000
N_Gen = Total - X_N.shape[0]
S_Gen = Total - X_S.shape[0]
V_Gen = Total - X_V.shape[0]
print (N_Gen, S_Gen, V_Gen)

n_batch = 512
latent_dim = 100
classes = 3
data = 186
np.random.seed(2)

(125561, 186, 1) (9305, 186, 1) (28954, 186, 1)
4439 120695 101046


In [11]:
model_name = 'Final/BLI/'

batch_no = '5023'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '6039'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '6049'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '6063'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

In [12]:
model_name = 'Final/BLIN/'

batch_no = '2053'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '2055'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '2057'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '3011'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '4069'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '5025'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '7057'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

batch_no = '8069'
combine_GAN_augment_data(model_name, batch_no, latent_dim, classes, data, X_N, X_S, X_V, N_Gen, S_Gen, V_Gen)

(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)
(392172, 186, 1) (392172,)


In [13]:
# batch_no = '5023'
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
# g_model.load_weights(model_name + str(batch_no) + '_cgan_generator.h5')
# [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
# z_input  = g_model.predict([z_input, labels_input], verbose=1)
# print (z_input.shape)

# show_plot(z_input, n_batch)

In [14]:
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
# g_model.load_weights(model_name + str(batch_no) + '_cgan_generator.h5')

# X_NN, y_NN = augment_using_GAN(g_model, X_N, latent_dim, N_Gen, n_batch, label=0)
# X_SS, y_SS = augment_using_GAN(g_model, X_S, latent_dim, S_Gen, n_batch, label=1)
# X_VV, y_VV = augment_using_GAN(g_model, X_V, latent_dim, V_Gen, n_batch, label=2)

# print (X_NN.shape, y_NN.shape, X_SS.shape, y_SS.shape, X_VV.shape, y_VV.shape)

# X = np.vstack((X_NN, X_SS, X_VV))
# y = np.hstack((y_NN, y_SS, y_VV))
# print (X.shape, y.shape)

# X_name = 'Data/GAN_GEN/' + batch_no + '_X'   
# y_name = 'Data/GAN_GEN/' + batch_no + '_y'   
# # print (X_name, y_name)
# np.save(X_name, X)
# np.save(y_name, y)